In [1]:
import kubernetes.client
import kubernetes.config
import kubernetes.watch
from kubernetes.client.rest import ApiException
import yaml
import tempfile
import json
import datetime

In [3]:
tf = tempfile.NamedTemporaryFile(mode="w")

kubernetes.config.KUBE_CONFIG_DEFAULT_LOCATION = tf.name
yaml.dump(config_dict, tf.file, default_flow_style=False)

kubernetes.config.load_kube_config()
api_client = kubernetes.client.ApiClient()
tf.close()

In [4]:
body = yaml.safe_load(open('../pyspark_scripts/stations/stations_spark_application.yaml'))
# custom_object_name = '-'.join([body['metadata']['name'], str(uuid.uuid4())[:8]])
# body['metadata']['name'] = custom_object_name

group, version = body["apiVersion"].split("/")
custom_object_name = body['metadata']['name']

custom_object_params = {
    "group": group,
    "version": version,
    "namespace": body['metadata']['namespace'],
    "plural": "sparkapplications",   
}

In [5]:
custom_objs_api = kubernetes.client.CustomObjectsApi(api_client)

In [41]:
custom_objs_api.delete_namespaced_custom_object(**custom_object_params, name=custom_object_name)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'stations-spark-script-6f885d3d',
  'group': 'sparkoperator.k8s.io',
  'kind': 'sparkapplications',
  'uid': '9f70c20f-a9b3-49d3-8cd4-f363e041268d'}}

In [ ]:
status = custom_objs_api.get_namespaced_custom_object_status(**custom_object_params, name=custom_object_name)
print(json.dumps(status['status'], indent=3))

In [15]:
watch = kubernetes.watch.Watch()

args = {
    'field_selector': "metadata.name=%s" % custom_object_name,
    **custom_object_params
}

resp = custom_objs_api.list_namespaced_custom_object(**args)
args['func'] = custom_objs_api.list_namespaced_custom_object
args['timeout_seconds'] = 25

# checking if there is any running object that match the new obj specification
if len(resp['items']):

    # deleting completed objects
    for event in watch.stream(**args):
        if event['type'] == 'DELETED' or not event['type']:
            watch.stop()

        elif 'status' in event['object']:
            state = event['object']['status']['applicationState']['state']

            if state in ['FAILED', 'SUBMISSION_FAILED', 'COMPLETED']:
                custom_objs_api.delete_namespaced_custom_object(**custom_object_params, name=custom_object_name)

# creating object
custom_objs_api.create_namespaced_custom_object(
    **custom_object_params,
    body=body
)

args['timeout_seconds'] = 480

for event in watch.stream(**args):
    print('Resource %s %s: %s' % (event['object']['metadata']['name'], event['type'], datetime.datetime.now().isoformat()))

    if 'status' in event['object']:
        print("New state reported: %s" % json.dumps(event['object']['status'], indent=2))

        if event['object']['status']['applicationState']['state'] in ['FAILED', 'SUBMISSION_FAILED', 'COMPLETED']:
            watch.stop()

            if event['object']['status']['applicationState']['state'] != 'COMPLETED':
                raise Exception("Spark application does not completed without errors")



Resource stations-spark-script ADDED: 2022-03-27T09:29:51.914976
Resource stations-spark-script MODIFIED: 2022-03-27T09:29:56.052677
New state reported: {
  "applicationState": {
    "state": "SUBMITTED"
  },
  "driverInfo": {
    "podName": "stations-spark-script-driver",
    "webUIAddress": "10.152.183.176:4040",
    "webUIPort": 4040,
    "webUIServiceName": "stations-spark-script-ui-svc"
  },
  "executionAttempts": 1,
  "lastSubmissionAttemptTime": "2022-03-27T12:29:56Z",
  "submissionAttempts": 1,
  "submissionID": "1e1ed5a7-0d59-4230-9a44-5efd9d0f7839",
  "terminationTime": null
}
Resource stations-spark-script MODIFIED: 2022-03-27T09:29:56.063330
New state reported: {
  "applicationState": {
    "state": "SUBMITTED"
  },
  "driverInfo": {
    "podName": "stations-spark-script-driver",
    "webUIAddress": "10.152.183.176:4040",
    "webUIPort": 4040,
    "webUIServiceName": "stations-spark-script-ui-svc"
  },
  "executionAttempts": 1,
  "lastSubmissionAttemptTime": "2022-03-27T12

In [2]:
config_dict = {
    "kind": "Config",
    "apiVersion": "v1",
    "preferences": {},
    "clusters": [
        {
            "name": "microk8s-cluster",
            "cluster": {
                "server": "https://127.0.0.1:16443",
                "certificate-authority-data": "LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUREekNDQWZlZ0F3SUJBZ0lVVzFtVmFYNjhXRmJGTGNzRW5Jc3FtQllMUkFzd0RRWUpLb1pJaHZjTkFRRUwKQlFBd0Z6RVZNQk1HQTFVRUF3d01NVEF1TVRVeUxqRTRNeTR4TUI0WERUSXhNRGd4T0RFNU1EYzFOMW9YRFRNeApNRGd4TmpFNU1EYzFOMW93RnpFVk1CTUdBMVVFQXd3TU1UQXVNVFV5TGpFNE15NHhNSUlCSWpBTkJna3Foa2lHCjl3MEJBUUVGQUFPQ0FROEFNSUlCQ2dLQ0FRRUFwNk5aaFYyUXRBM29uaGMxK05FRmFxTGJHYkd2ckpkV01zTWsKOVJnN1ZtMG85Mkh6cGhRakt0YW1xZGFMMHpISy82L3Y0eS9IMGxwMTBmYW03NUNjbjl3SUNBcTlIdlBtQ3BRQgppbEUxdUZhcTNLV2tBUm9qellhODh6b1lJWU1HbVd2enBiUGZrckxKeUU4amp5QzFsOTFwTWpZS21tcEwxZm9GClQyQkNNMTE0YzU2NTA2TU1paGxmRDlMVE9tQjl2eEJUTHpWczZTZ3grd2E4ZnFCUzJRcFA5dXJOdm52SnF5V04KNHJ2SHJWeUROZ2Y0OG9OU2tIeHFLRVRMQS9KY0dDMjcrU2tGZHdndHhhWTJWL1FnRklCZEIwUU1JcmljWWxXSwpycnlpVXEwZUJOZ2lJRERESERpRWJKZkZzakdPcWVIeGlxcmRrV0lIZlplYTFBV3EyUUlEQVFBQm8xTXdVVEFkCkJnTlZIUTRFRmdRVWtKaE1rRHB6Wk96dVVEdnlhTEVnWER0ZE5ENHdId1lEVlIwakJCZ3dGb0FVa0poTWtEcHoKWk96dVVEdnlhTEVnWER0ZE5ENHdEd1lEVlIwVEFRSC9CQVV3QXdFQi96QU5CZ2txaGtpRzl3MEJBUXNGQUFPQwpBUUVBVnVkaDdaUWoxaks1K1JCbjVrdGNLSUM5eS9SVDQveHlhc1BrdFN5M0ZTMFdhSTBucGtwNXZtWEw3MkNNCkEveW5GQ3BieDNFemM2a1JmT1pYQ2E5Y0d5SUlTNDBFeSsranRiYzQ0d1NUN0ZlYmRpTktGMjNlc2Z2aTNSa1QKb25ybjlNbThqZDNacElycU9RS2JtUWxZbm1ITExPUjZwWGpadDBMUEJwaEdzVVFrZWsrR2NoYXFqd2ZBWDRwZAp6Q3ZpNUY5WE1Ub25LTEJYN2RVSUhpWmliVS9ndE94TnIzMEtOQTBSQ1pqRVFLdGFMQkEwQVB5c3V4TitMYVJICnZRK2dQdWViZFdobjM3YmQwRmNIaHpZeklvYlJYbktjNWtnRi8reFhsTlFDVWdHaTcwUmpkT1A5RlhYSGgzODMKREEyei84SnRkSmJGSUl3NWhJQW42U3VId0E9PQotLS0tLUVORCBDRVJUSUZJQ0FURS0tLS0tCg=="
            }
        }
    ],
    "users": [
        {
            "name": "admin",
            "user": {
                "token": "OHVYSWhabXlFdHNwbnhYKzV6N0FoSmJ4SEs5OWR3ektybE5ETXRYQUQ0UT0K"
            }
        }
    ],
    "contexts": [
        {
            "name": "microk8s",
            "context": {
                "cluster": "microk8s-cluster",
                "user": "admin"
            }
        }
    ],
    "current-context": "microk8s"
}